In [1]:
import pandas as pd
df['readid'].head()

In [14]:
def load_training_data():
    df = pd.read_csv('numerai_training_data.csv')
    return df
    

In [15]:
def load_ticker_data():
    tickers = pd.read_csv('latest.csv')
    return tickers

In [3]:
df = load_training_data()

In [4]:
df.columns

Index(['readid', 'era', 'data_type', 'feature_intelligence1',
       'feature_intelligence2', 'feature_intelligence3',
       'feature_intelligence4', 'feature_intelligence5',
       'feature_intelligence6', 'feature_intelligence7',
       ...
       'feature_wisdom38', 'feature_wisdom39', 'feature_wisdom40',
       'feature_wisdom41', 'feature_wisdom42', 'feature_wisdom43',
       'feature_wisdom44', 'feature_wisdom45', 'feature_wisdom46', 'target'],
      dtype='object', length=314)

In [8]:
len(df['readid'].unique())


501808

## There are 501808 unique ids in the training data.

This means that they do not uniquely refer to a stock. You cannot take the id and use it to determine what stock it is. 

## Numerai Signals 

- Dont work on this. Just focus on the base numerAI Trournament

In [16]:
tickers = load_ticker_data()

In [20]:
tickers['bloomberg_ticker'].head()

0    SVW AU
1    GEM AU
2    VOC AU
3    AZJ AU
4    MLD AU
Name: bloomberg_ticker, dtype: object

In [ ]:
num_tickers = len(tickers['bloomberg_ticker'].unique())
print(f'There are currently {num_tickers} in the signals universe')

There are currently 5433 in the signals universe
